# Preprocessing
@Author: Bruno Vieira

Goals: Perform data acquisition and investigate the data in order to clean It.

In [211]:
# Libs
import os
import json
import importlib

import numpy as np
import pandas as pd
import utils.dev.cleaning as cl

from tqdm import tqdm
from operator import itemgetter

pd.set_option('display.max_columns', 100)

# Data Acquisition

In [212]:
# Paths and Filenames
DATA_INPUT_PATH = 'data/raw'
DATA_OUTPUT_PATH = 'data/interim'
DATA_INPUT_NAME1 = 'support.json'
DATA_INPUT_NAME2 = 'dev.json'
DATA_INPUT_NAME3 = 'train.json'
DATA_INPUT_NAME4 = 'test.json'
DATA_OUTPUT_NAME1 = 'dev.csv'
DATA_OUTPUT_NAME2 = 'train.csv'
DATA_OUTPUT_NAME3 = 'test.csv'

In [213]:
# Data
with open(os.path.join('..',DATA_INPUT_PATH, DATA_INPUT_NAME2)) as file_json:
    data_twitter_dev = json.load(file_json)
with open(os.path.join('..',DATA_INPUT_PATH, DATA_INPUT_NAME3)) as file_json:
    data_twitter_train = json.load(file_json)
with open(os.path.join('..',DATA_INPUT_PATH, DATA_INPUT_NAME4)) as file_json:
    data_twitter_test = json.load(file_json)

In [214]:
pd.DataFrame.from_records(pd.DataFrame.from_records(data_twitter_train)['profile'])

,id,id_str,name,screen_name,location,profile_location,description,url,entities,protected,followers_count,friends_count,listed_count,created_at,favourites_count,utc_offset,time_zone,geo_enabled,verified,statuses_count,lang,contributors_enabled,is_translator,is_translation_enabled,profile_background_color,profile_background_image_url,profile_background_image_url_https,profile_background_tile,profile_image_url,profile_image_url_https,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,has_extended_profile,default_profile,default_profile_image
0,17461978,17461978,SHAQ,SHAQ,"Orlando, FL","{'id': '55b4f9e5c516e0b6', 'url': 'https://api...","VERY QUOTATIOUS, I PERFORM RANDOM ACTS OF SHAQ...",https://t.co/7hsiK8cCKW,{'url': {'urls': [{'url': 'https://t.co/7hsiK8...,False,15349596,692,45568,Tue Nov 18 10:27:25 +0000 2008,142,None,None,True,True,9798,None,False,False,False,080203,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg....,False,http://pbs.twimg.com/profile_images/1673907275...,https://pbs.twimg.com/profile_images/167390727...,2FC2EF,181A1E,252429,666666,True,False,False,False
1,1297437077403885568,1297437077403885568,Jennifer Fishpaw,JenniferFishpaw,,None,,None,{'description': {'urls': []}},False,0,44,0,Sun Aug 23 07:35:11 +0000 2020,7,None,None,False,False,0,None,False,False,False,F5F8FA,None,None,False,http://pbs.twimg.com/profile_images/1297437406...,https://pbs.twimg.com/profile_images/129743740...,1DA1F2,C0DEED,DDEEF6,333333,True,True,True,False
2,17685258,17685258,Brad Parscale,parscale,Florida,None,Owner @ Parscale Strategy. Senior Advisor Digi...,https://t.co/GooZcYDqFg,{'url': {'urls': [{'url': 'https://t.co/GooZcY...,False,762839,475,3201,Thu Nov 27 18:47:32 +0000 2008,953,None,None,True,True,5518,None,False,False,False,1A1B1F,http://abs.twimg.com/images/themes/theme9/bg.gif,https://abs.twimg.com/images/themes/theme9/bg....,False,http://pbs.twimg.com/profile_images/1295453225...,https://pbs.twimg.com/profile_images/129545322...,AB2316,FFFFFF,FFFFFF,666666,False,False,False,False
3,15750898,15750898,FOX 13 Tampa Bay,FOX13News,"Tampa, FL",None,Bringing you the important stuff like breaking...,https://t.co/RtP9QYEZCq,{'url': {'urls': [{'url': 'https://t.co/RtP9QY...,False,327587,4801,1744,Wed Aug 06 15:12:10 +0000 2008,2946,None,None,True,True,192876,None,False,False,False,00064A,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg....,False,http://pbs.twimg.com/profile_images/1293193013...,https://pbs.twimg.com/profile_images/129319301...,0B2F8A,FFFFFF,E8EEF0,333333,True,False,False,False
4,1659167666,1659167666,Vonte The Plug 🎤🔌,VonteThePlugNC,"Jacksonville Beach, FL","{'id': '5e281c17a74c170f', 'url': 'https://api...",MOTIVATION 3 OUT NOW 🔥 Singles: ‘Lil Shawdy’ &...,https://t.co/5cY8GWvk8E,{'url': {'urls': [{'url': 'https://t.co/5cY8GW...,False,13324,647,44,Sat Aug 10 03:25:35 +0000 2013,729,None,None,False,False,103,None,False,False,False,C0DEED,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg....,False,http://pbs.twimg.com/profile_images/1181662400...,https://pbs.twimg.com/profile_images/118166240...,1DA1F2,C0DEED,DDEEF6,333333,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8273,1630890068,1630890068,Mike Waters,MikeWatersSYR,,None,College basketball writer at https://t.co/OOrF...,None,{'description': {'urls': [{'url': 'https://t.c...,False,10997,821,221,Mon Jul 29 17:46:44 +0000 2013,20115,None,None,False,True,25790,None,False,False,False,C0DEED,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg....,False,http://pbs.twimg.com/profile_images/6543578675...,https://pbs.twimg.com/profile_images/654357867...,1DA1F2,C0DEED,DDEEF6,333333,True,Fals

In [215]:
df_twitter_train.columns

Index(['id', 'id_str', 'name', 'screen_name', 'location', 'profile_location',
       'description', 'followers_count', 'friends_count', 'listed_count',
       'created_at', 'favourites_count', 'geo_enabled', 'verified',
       'statuses_count', 'is_translation_enabled', 'profile_background_color',
       'profile_background_image_url', 'profile_background_image_url_https',
       'profile_background_tile', 'profile_image_url',
       'profile_image_url_https', 'profile_link_color',
       'profile_sidebar_border_color', 'profile_sidebar_fill_color',
       'profile_text_color', 'profile_use_background_image',
       'has_extended_profile', 'default_profile', 'default_profile_image'],
      dtype='object')

In [219]:
# function to create tabular data from json
def create_tabular_data(dataset):
    df = pd.DataFrame.from_records(pd.DataFrame.from_records(dataset)['profile'])
    df['label'] = pd.DataFrame.from_records(pd.DataFrame.from_records(dataset)['label'])
    return df

In [220]:
# recovering data as a table
df_twitter_dev = create_tabular_data(data_twitter_dev)
df_twitter_train = create_tabular_data(data_twitter_train)
df_twitter_test = create_tabular_data(data_twitter_test)

# 2) Data Cleaning

- Types Identification
- Duplicates 
- Quasi-Constant Filter 
- Missing Values 
- Fill strategy

## 2.1) Types Identification

In [222]:
column_types = cl.check_dtypes(df_twitter_dev)

In [223]:
column_types

{'object': ['id',
  'id_str',
  'name',
  'screen_name',
  'location',
  'profile_location',
  'description',
  'url',
  'entities',
  'protected',
  'followers_count',
  'friends_count',
  'listed_count',
  'created_at',
  'favourites_count',
  'utc_offset',
  'time_zone',
  'geo_enabled',
  'verified',
  'statuses_count',
  'lang',
  'contributors_enabled',
  'is_translator',
  'is_translation_enabled',
  'profile_background_color',
  'profile_background_image_url',
  'profile_background_image_url_https',
  'profile_background_tile',
  'profile_image_url',
  'profile_image_url_https',
  'profile_link_color',
  'profile_sidebar_border_color',
  'profile_sidebar_fill_color',
  'profile_text_color',
  'profile_use_background_image',
  'has_extended_profile',
  'default_profile',
  'default_profile_image',
  'label']}

All data types are object. In practise We should treat some of the columns as integers, because of their nature. So, let's create a function that converts a list of columns to specified types of data.

In [224]:
def convert_types(df=None, type_cast='int', list_columns=None):
    for col in list_columns:
        df = df.astype({col:type_cast})
    return df

In [225]:
list_convert_integer = ['followers_count', 'friends_count', 'listed_count', 'favourites_count']
df_twitter_dev = convert_types(df_twitter_dev, type_cast='int', list_columns=list_convert_integer)
df_twitter_train = convert_types(df_twitter_train, type_cast='int', list_columns=list_convert_integer)
df_twitter_test = convert_types(df_twitter_test, type_cast='int', list_columns=list_convert_integer)

## 2.2) Duplicates

The author provided these 3 datasets, so We believe they don't share users. This step can be skipped.

## 2.3) Quasi-Constant Filter

Columns with constant values and with the mode with extremely high frequency will be droped.

In [226]:
importlib.reload(cl)

<module 'utils.dev.cleaning' from '/Users/bruno.santos/Desktop/Mestrado/projects/bot_detection_twitter_profile_features/utils/dev/cleaning.py'>

In [227]:
constant_columns, quasi_constant_columns = cl.check_constant_columns(df=df_twitter_dev, threshold=0.99)

Analysing Constant Columns
5 Constant Columns Found
['utc_offset', 'time_zone', 'lang', 'contributors_enabled', 'is_translator']
6 Quasi-Constant Columns Found
['protected', 'utc_offset', 'time_zone', 'lang', 'contributors_enabled', 'is_translator']


In [228]:
# drop constant and quasi-constant columns
columns_to_drop_constant = constant_columns+quasi_constant_columns
df_twitter_dev.drop(columns_to_drop_constant, axis=1, inplace=True)
df_twitter_train.drop(columns_to_drop_constant, axis=1, inplace=True)
df_twitter_test.drop(columns_to_drop_constant, axis=1, inplace=True)

## 2.4) Missing Values

Explicit missing values (None and np.nan). As expected, But maybe missing are encoded as other formats, such as -999, 999, NA, NaN, None, all as either strings or numbers. Let's try to find some of them.

In [229]:
list_of_missing = [-999, -1, '-999', '9999', 'None', 'NaN', 'nan', 'NAN', 'NA', ' ', '', 'None ']
for missing in list_of_missing:
    df_twitter_train.replace({missing:np.nan}, inplace=True)
    df_twitter_test.replace({missing:np.nan}, inplace=True)
    df_twitter_dev.replace({missing:np.nan}, inplace=True)


In [230]:
series_missing = cl.check_missing(df_twitter_train, list_of_missing=list_of_missing)

In [231]:
series_missing

profile_location                      0.912056
url                                   0.513409
location                              0.290288
profile_background_image_url_https    0.263711
profile_background_image_url          0.263711
description                           0.129621
profile_sidebar_border_color          0.000000
profile_background_tile               0.000000
profile_image_url                     0.000000
profile_image_url_https               0.000000
profile_link_color                    0.000000
id                                    0.000000
profile_sidebar_fill_color            0.000000
profile_text_color                    0.000000
has_extended_profile                  0.000000
default_profile                       0.000000
default_profile_image                 0.000000
profile_use_background_image          0.000000
statuses_count                        0.000000
profile_background_color              0.000000
is_translation_enabled                0.000000
id_str       

## 2.5) Fill Strategy

Note that location has a large amount of missing. But, We can easily handle these missing, using the information of "missingness" to train the model. Besides that, description can be tackled the same way and friends_count and listed_count can be filled with 0.

# 3) Saving the Data

Before We finish this step, I would drop some useless columns. All others will be feature engineered and evaluated as predictors in the step of Exploratory Data Analysis, Feature Engineering and Feature Selection

In [232]:
df_twitter_train.columns

Index(['id', 'id_str', 'name', 'screen_name', 'location', 'profile_location',
       'description', 'url', 'entities', 'followers_count', 'friends_count',
       'listed_count', 'created_at', 'favourites_count', 'geo_enabled',
       'verified', 'statuses_count', 'is_translation_enabled',
       'profile_background_color', 'profile_background_image_url',
       'profile_background_image_url_https', 'profile_background_tile',
       'profile_image_url', 'profile_image_url_https', 'profile_link_color',
       'profile_sidebar_border_color', 'profile_sidebar_fill_color',
       'profile_text_color', 'profile_use_background_image',
       'has_extended_profile', 'default_profile', 'default_profile_image',
       'label'],
      dtype='object')

In [233]:
columns_to_drop_useless = ['url', 'entities']
df_twitter_train.drop(columns_to_drop_useless, axis=1, inplace=True)
df_twitter_dev.drop(columns_to_drop_useless, axis=1, inplace=True)
df_twitter_test.drop(columns_to_drop_useless, axis=1, inplace=True)

In [234]:
# exporting final data to the interim folder
df_twitter_dev.to_csv(os.path.join('..', DATA_OUTPUT_PATH, DATA_OUTPUT_NAME1), index=False)
df_twitter_train.to_csv(os.path.join('..', DATA_OUTPUT_PATH, DATA_OUTPUT_NAME2), index=False)
df_twitter_test.to_csv(os.path.join('..', DATA_OUTPUT_PATH, DATA_OUTPUT_NAME3), index=False)